<a href="https://colab.research.google.com/github/crimsoneer/Twython_Streamer_test/blob/primary/analysis_visu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'll use this file to analyse and visualise my collected cleaned output

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

style.use('seaborn') #sets the size of the charts
style.use('ggplot')
all_data = pd.read_csv("all_data.csv", encoding= "ISO-8859-1")
cleaned_data = pd.read_csv("cleaned_data.csv", encoding= "ISO-8859-1")
drug_dict = pd.read_csv("drugs.csv", encoding= "ISO-8859-1")
no_duplicates = all_data.drop_duplicates(subset="unique_ref", keep="last")
clean_no_duplicates = cleaned_data.drop_duplicates(subset="unique_ref", keep="last")

print("Total count:")
#this is every single row, igoring how clean it is or whether it's duplicated
print(all_data.shape)
print("Clean data:")
#this is only data where I can estimate price
print(cleaned_data.shape)
print("Total Individual Adverts:")
#all the data, but without any duplicates (for instance, ad run multiple days)
print(no_duplicates.shape)
print("Clean Individual Adverts:")
#only single adverts with an extracted or advertised price
print(clean_no_duplicates.shape)

clean_no_duplicates.head()

Total count:
(10930, 36)
Clean data:
(6193, 36)
Total Individual Adverts:
(3149, 36)
Clean Individual Adverts:
(1800, 36)


,Unnamed: 0,index,Ã¯Â»Â¿web-scraper-order,web-scraper-start-url,name,description,price,seller,from,to,sold,link,link-href,date,unique_ref,seller name,seller reviews,seller score,Extracted views,Extracted sold,price listed,Currency,Volume,Unit of Volume,Price Per Unit,substance,extracted name,name confidence,pill quantity,price per pill,cloned,extracted weight,extracted weight unit,extracted weight number,weight in g,price per gram
0,1,1,1587452527-12696,http://paxromanadfudhte.onion/category/1?page=101,Cloned BUY COCAINE POWDER (99% PURE),"Cocaine, also known as coke, is a strong stimu...",20.00,supremepharma(4)\n(5.00),USA,WW,Views: 0\nSold: 0,Escrow \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\...,http://paxromanadfudhte.onion/listing/Q03B45rX...,2020-04-21,"Cloned BUY COCAINE POWDER (99% PURE)Cocaine, a...",supremepharma,4,5.0,0,0,1 gram \n for 20.00 USD,USD,1.0,gram,20.00,COCAINE,COCAINE,100.0,NaN,NaN,True,NaN,NaN,NaN,1.000,20.000000
10,14,14,1587452921-13635,http://paxromanadfudhte.onion/category/1?page=7,Best QualityAlprazolam Xanax 2mg Grade Xanax Bars,Good quality products. Good and affordable pri...,450.00,usdrill(8)\n(5.00),USA,WW,Views: 53\nSold: 0,Escrow \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\...,http://paxromanadfudhte.onion/listing/ovXjqm6p...,2020-04-21,Best QualityAlprazolam Xanax 2mg Grade Xanax B...,usdrill,8,5.0,53,0,120 gram \n for 450.00 USD,USD,120.0,gram,3.75,BENZOS,Xanax,100.0,NaN,NaN,False,2mg,mg,2.0,120.000,3.750000
19,24,24,1587452782-13460,http://paxromanadfudhte.onion/category/1?page=25,ÃÂ£15 RELAUNCH SALE! 20 x 3mg Xanax GREEN HUL...,After a short hiatus we are back with a new a ...,9.95,BenzoBearUK(10)\n(5.00),GB,GB,Views: 27\nSold: 0,Escrow \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\...,http://paxromanadfudhte.onion/listing/R95281P3...,2020-04-21,ÃÂ£15 RELAUNCH SALE! 20 x 3mg Xanax GREEN HUL...,BenzoBearUK,10,5.0,27,0,9.95 USD,USD,3.0,mg,9.95,BENZOS,Xanax,100.0,20.0,0.4975,False,3mg,mg,3.0,0.003,3316.666667
20,25,25,1587452462-12426,http://paxromanadfudhte.onion/category/1?page=128,Cloned BUY SUBUTEX 8MG,Subutex is used to treat dependence/addiction ...,8.00,supremepharma(4)\n(5.00),USA,WW,Views: 1\nSold: 0,Escrow \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\...,http://paxromanadfudhte.onion/listing/qv20o5lX...,2020-04-21,Cloned BUY SUBUTEX 8MGSubutex is used to treat...,supremepharma,4,5.0,1,0,8.00 USD,USD,8.0,MG,8.00,PRESCRIPTION OPIOIDS,SUBUTEX,100.0,NaN,NaN,True,8MG,MG,8.0,0.008,1000.000000
25,37,37,1587452366-12005,http://paxromanadfudhte.onion/category/1?page=170,Cloned 1 GRAM HIGH HEAT COLUMBIAN COCAINE HIGH...,THERE HAS BEEN A REASON THAT WE HAVE STAYED AW...,100.00,LabTested(1)\n(5.00),USA,WW,Views: 1\nSold: 0,No Escrow \t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t...,http://paxromanadfudhte.onion/listing/7n2bezJX...,2020-04-21,Cloned 1 GRAM HIGH HEAT COLUMBIAN COCAINE HIGH...,LabTested,1,5.0,1,0,1 gram \n for 100.00 USD,USD,1.0,gram,100.00,COCAINE,COCAINE,100.0,NaN,NaN,True,1 GRAM,GRAM,1.0,1.000,100.000000


In [0]:
seller_columns = ["price", "from", "to", "seller name", "seller score", "seller reviews", "Extracted views", "Extracted sold", "substance", "cloned"]
seller_df = no_duplicates[seller_columns].copy()

seller_df["from"] = seller_df["from"].astype('category')
seller_df["to"] = seller_df["to"].astype('category')
seller_df["substance"] = seller_df["substance"].astype('category')
seller_df["price"] = seller_df["price"].astype('float')
seller_df["reviews"] = seller_df["seller score"].fillna(0).astype('int')
seller_df["sold"] = seller_df["Extracted sold"].fillna(0).astype('int')
seller_df["views"] = seller_df["Extracted views"].fillna(0).astype('int')
seller_df["cloned"] = seller_df["Extracted views"].fillna(0).astype('bool')

In [0]:
seller_df

,price,from,to,seller name,seller score,seller reviews,Extracted views,Extracted sold,substance,cloned,reviews,sold,views
0,20.00,USA,WW,supremepharma,5.0,4,0,0,PRESCRIPTION OPIOIDS,False,5,0,0
1,20.00,USA,WW,supremepharma,5.0,4,0,0,COCAINE,False,5,0,0
14,450.00,USA,WW,usdrill,5.0,8,53,0,BENZOS,True,5,0,53
24,9.95,GB,GB,BenzoBearUK,5.0,10,27,0,BENZOS,True,5,0,27
25,8.00,USA,WW,supremepharma,5.0,4,1,0,PRESCRIPTION OPIOIDS,True,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10925,310.00,USA,WW,UStoUS,5.0,3,8,0,COCAINE,True,5,0,8
10926,190.00,USA,USA,DaDankStore,5.0,0,59,0,MARIJUANA,True,5,0,59
10927,350.00,USA,WW,pharmawebking2,5.0,1,12,0,PRESCRIPTION OPIOIDS,True,5,0,12
10928,2.65,NL,WW,Wvanoranje,5.0,46,364,14,MESCALINE,True,5,14,364


In [0]:
def count_of_unique(x):
    list_of_uniques = np.unique(x)
    return len(list_of_uniques)

seller_list = seller_df.groupby(["seller name"]).agg(
    average_price = ("price", np.mean),
    review_score = ("reviews", np.mean),
    advert_count = ("substance", len),
    total_views = ("views", sum),
    total_sales = ("sold", sum),
    substances_sold= ("substance", count_of_unique),
    cloned_count = ("cloned", sum)
)

In [0]:
'''def new_views(x):
    total_views = sum(x)
    first_view = x[0]
    new_count = total_views - first_view
    return new_count

def first_view(x):
    total_views = sum(x)
    first_view_count = x[0]
    return first_view_count'''

unique_sales = all_data.groupby(["unique_ref", "substance", "price"]).agg(
    days_run = ("name", len),
    total_views = ("Extracted views", sum),
    first_view_count = ("Extracted views", min),
    total_sales = ("Extracted sold", sum),
    first_sales_count = ("Extracted sold", min)
)

unique_sales["new_sales"] = unique_sales["total_sales"] - unique_sales["first_sales_count"]
unique_sales["sales_per_day"] = unique_sales["new_sales"] / unique_sales["days_run"]
unique_sales["new_views"] = unique_sales["total_views"] - unique_sales["first_view_count"]
unique_sales["views_per_day"] = unique_sales["new_views"] / unique_sales["days_run"]

unique_sales = unique_sales.reset_index()

unique_sales.head()

,unique_ref,substance,price,days_run,total_views,first_view_count,total_sales,first_sales_count,new_sales,sales_per_day,new_views,views_per_day
0,#0.5GR #SALE || COKE COLOMBIAN 91% || USA AUS ...,COCAINE,37.70,1,150,150,2,2,0,0.0,0,0.0
1,#0.5GR #SALE || COKE COLOMBIAN 91% || USA AUS ...,COCAINE,37.80,1,152,152,2,2,0,0.0,0,0.0
2,#0.5GR #SALE || COKE COLOMBIAN 91% || USA AUS ...,COCAINE,37.93,1,142,142,2,2,0,0.0,0,0.0
3,#0.5GR #SALE || COKE COLOMBIAN 91% || USA AUS ...,COCAINE,38.01,1,127,127,2,2,0,0.0,0,0.0
4,#0.5GR #SALE || COKE COLOMBIAN 91% || USA AUS ...,COCAINE,38.03,1,149,149,2,2,0,0.0,0,0.0


In [0]:
unique_sales.describe()

,price,days_run,total_views,first_view_count,total_sales,first_sales_count,new_sales,sales_per_day,new_views,views_per_day
count,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000,6048.000000
mean,608.685488,1.807209,47.342427,28.211640,0.494544,0.343419,0.151124,0.036384,19.130787,4.458014
std,2835.480950,1.465168,125.342766,65.157741,2.396866,1.319212,1.659839,0.402746,90.790089,22.291000
min,0.110000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31.080000,1.000000,10.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,120.000000,1.000000,21.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,377.020000,2.000000,42.000000,29.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
max,168735.000000,6.000000,3626.000000,1203.000000,61.000000,20.000000,52.000000,13.000000,2864.000000,716.000000


In [0]:
substance_daily_breakdown = unique_sales.groupby(["substance"]).mean()


In [0]:
substance_daily_breakdown


,price,days_run,total_views,first_view_count,total_sales,first_sales_count,new_sales,sales_per_day,new_views,views_per_day
substance,,,,,,,,,,
ADDERALL,247.784646,3.383838,68.121212,18.656566,0.282828,0.070707,0.212121,0.045118,49.464646,11.360774
AMPHETAMINE,468.097149,1.154971,36.299708,32.353801,0.507310,0.454678,0.052632,0.015789,3.945906,1.070736
ANABOLIC STEROIDS,290.218261,3.173913,66.869565,25.913043,0.000000,0.000000,0.000000,0.000000,40.956522,8.390580
BENZOS,197.576688,1.870488,50.411890,31.074310,0.520170,0.409766,0.110403,0.026115,19.337580,4.520524
COCAINE,1267.894970,1.983234,53.859880,30.857485,0.585629,0.356886,0.228743,0.050878,23.002395,5.230020
FENTANYL,370.634853,3.970588,53.544118,8.558824,0.235294,0.044118,0.191176,0.042647,44.985294,10.018627
GHB,183.236997,1.177843,29.746356,26.026239,0.303207,0.297376,0.005831,0.002915,3.720117,0.895578
HEROIN,762.867247,2.505618,62.679775,21.252809,0.629213,0.241573,0.387640,0.080712,41.426966,8.650749
KETAMINE,1186.505962,1.781250,31.096154,17.526442,0.276442,0.240385,0.036058,0.007212,13.569712,2.944591


In [0]:
fig = px.bar(substance_daily_breakdown, x=substance_daily_breakdown.index, y='views_per_day')
fig.update_layout(title="Mean Daily Views per Substance")
fig.show()

In [0]:
fig = px.bar(substance_daily_breakdown, x=substance_daily_breakdown.index, y='sales_per_day')
fig.update_layout(title="Mean Daily Sales per Substance")
fig.show()

In [0]:
seller_list.describe()

,average_price,review_score,advert_count,total_views,total_sales,substances_sold,cloned_count
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,401.183842,4.905263,33.147368,829.852632,8.031579,4.705263,32.578947
std,661.234784,0.328559,63.144207,1385.492638,20.013002,3.280742,60.715342
min,2.666667,3.000000,1.000000,5.000000,0.000000,1.000000,1.000000
25%,71.264167,5.000000,6.500000,117.000000,0.000000,2.000000,6.500000
50%,240.000000,5.000000,14.000000,367.000000,1.000000,4.000000,14.000000
75%,451.106021,5.000000,32.500000,784.500000,6.500000,7.000000,32.500000
max,5407.731343,5.000000,404.000000,6954.000000,136.000000,14.000000,381.000000


In [0]:
fig = px.scatter(seller_list, x="average_price", y="total_sales")
fig.update_layout(title="Market Sellers, Average Price vs Total Sales")
fig.show()

In [0]:
fig = px.scatter_3d(seller_list, x="average_price", y="total_sales", z="advert_count", color="substances_sold", hover_name=seller_list.index, hover_data=["total_sales", "advert_count", "average_price", "substances_sold", "cloned_count"])
fig.update_layout(title="Market Sellers, Average Price, Total Sale, and Count of Substance Sold")
fig.show()

'''username = 'HIDDEN'
api_key = 'HIDDEN'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)'''

py.plot(fig, filename = 'seller_scatter', auto_open=True)

NameError: ignored

We've identified some seriously big players.  I wonder if any of them knew the market was going to die off?
Why do have so many cloned adverts?  And we can we use average daily views to estimate traffic?

In [0]:
seller_list = seller_list.dropna(axis=0)
seller_list

In [0]:
aggregated_total = all_data.copy()
aggregated_clean = cleaned_data.copy()

to_aggregate = ["SALVIA DIVINORUM", "PCP AND ANALOGS", "MESCALINE", "Fentanyl", "GHB","HEROIN","LSD","OPIUM","PSILOCYBIN","FLUNITRAZEPAM"]
aggregate_total_filter = aggregated_total["substance"].isin(to_aggregate)
aggregated_total.loc[aggregate_total_filter, "substance"]="other"

aggregate_clean_filter = aggregated_clean["substance"].isin(to_aggregate)
aggregated_clean.loc[aggregate_clean_filter, "substance"]="other"

That is a lot of pills.


In [0]:
paired = sns.color_palette("Paired", 12)
with paired:
    aggregated_clean.pivot_table(values="price", index="date", columns="substance",aggfunc=len).plot.area(figsize=[15,15], title="DarkNet Market Substance Breakdown")

In [0]:
#same thing but using plotly instead of matplotlib
substances_breakdown = cleaned_data.groupby(["date", "substance"]).agg(
    total_listings = ("unique_ref", len)
)
substances_breakdown = substances_breakdown.sort_values(by=["total_listings"], ascending=True)
substances_breakdown.reset_index(inplace=True)

In [0]:
df = px.data.gapminder()
fig = px.area(substances_breakdown, x="date", y="total_listings", color="substance")
fig.update_layout(title="Count of Advert by Substance")
fig.show()

Let's look at originating countries - where do the drugs come from and go to?

In [0]:
all_data.pivot_table(values="unique_ref", index="date", columns="from",aggfunc=len).plot.area(figsize=[15,15], title="Count of Advert by Originating Country")

In [0]:
originating_countries = all_data.groupby(["from","date"]).agg(
    count_of_advert = ("unique_ref", len)
)
originating_countries = originating_countries.sort_values(by=["count_of_advert"], ascending=True)
originating_countries = originating_countries.reset_index()
list_of_countries = all_data["from"].unique()

In [0]:
#replicating using plotly
df = px.data.gapminder()
fig = px.area(originating_countries, x="date", y="count_of_advert", color="from")
fig.update_layout(title="Count of Advert by Originating Country")
fig.show()

In [0]:
fig = make_subplots(rows=4, cols=3)
x_loc=1
y_loc=1
originating_countries = originating_countries.sort_values(by=["date"])

for value in list_of_countries:
    drug_name = originating_countries["from"] == value
    fig.add_trace(
        go.Scatter(x=originating_countries[drug_name]["date"], y=originating_countries[drug_name]["count_of_advert"],
    mode="lines",
    name=value)
        , col=x_loc, row=y_loc)
    x_loc +=1
    if x_loc==4:
        y_loc+=1
        x_loc=1
fig.update_layout(title="Number of Adverts by Originating Country", width=800)
fig.show()

What about the breakdown of UK drugs?

In [0]:
from_uk = all_data["from"] == "GB"
to_uk =  all_data["to"] == "GB"
from_usa = all_data["from"] == "USA"

In [0]:
with paired:
    aggregated_total[from_uk].pivot_table(values="price", index="date", columns="substance",aggfunc=len).plot.area(figsize=[15,15], title="Adverts from the UK Substance Breakdown")

In [0]:
from_uk = cleaned_data["from"] == "GB"
uk_substances = cleaned_data[from_uk].groupby(["date", "substance"]).agg(
    total_listings = ("unique_ref", len)
)
uk_substances= uk_substances.sort_values(by="total_listings")
uk_substances.reset_index(inplace=True)

In [0]:
df = px.data.gapminder()
fig = px.area(uk_substances, x="date", y="total_listings", color="substance")
fig.update_layout(title="Count of Adverts Originating from UK, by Substance")
fig.show()


What about the US?

In [0]:
from_US = cleaned_data["from"] == "USA"
US_substances = cleaned_data[from_US].groupby(["date", "substance"]).agg(
    total_listings = ("unique_ref", len)
)
US_substances = US_substances.sort_values(by=["total_listings"], ascending=True)
US_substances.reset_index(inplace=True)

from_NL = cleaned_data["from"] == "NL"
NL_substances = cleaned_data[from_NL].groupby(["date", "substance"]).agg(
    total_listings = ("unique_ref", len)
)
NL_substances = US_substances.sort_values(by=["total_listings"], ascending=True)
NL_substances.reset_index(inplace=True)

from_DE = cleaned_data["from"] == "DE"
DE_substances = cleaned_data[from_DE].groupby(["date", "substance"]).agg(
    total_listings = ("unique_ref", len)
)
DE_substances = DE_substances.sort_values(by=["total_listings"], ascending=True)
DE_substances.reset_index(inplace=True)


In [0]:
df = px.data.gapminder()
fig = px.area(US_substances, x="date", y="total_listings", color="substance")
fig.update_layout(title="Count of Adverts Originating from US, by Substance")
fig.show()

In [0]:
df = px.data.gapminder()
fig = px.area(NL_substances, x="date", y="total_listings", color="substance")
fig.update_layout(title="Count of Adverts Originating from Netherlands, by Substance")
fig.show()

In [0]:
df = px.data.gapminder()
fig = px.area(DE_substances, x="date", y="total_listings", color="substance")
fig.update_layout(title="Count of Adverts Originating from Germany, by Substance")
fig.show()


In [0]:
with paired:
    aggregated_total[from_usa].pivot_table(values="price", index="date", columns="substance",aggfunc=len).plot.area(figsize=[15,15], title="Adverts from the USA Substance Breakdown")

In [0]:
drug_df = clean_no_duplicates.groupby(["substance"]).agg(
    mean_advert_price = ("price", np.mean),
    mean_price_per_gram = ("price per gram", np.mean),
    mean_price_per_pill = ("price per pill", np.mean),
    median_advert_price = ("price", np.median),
    median_price_per_gram = ("price per gram", np.median),
    median_price_per_pill = ("price per pill", np.median),
    grams_per_sale = ("weight in g", sum),
    count_of_ads = ("unique_ref", len),
    count_of_cloned = ("cloned", sum))

In [0]:
drug_df


In [0]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

list_of_substances = cleaned_data["substance"].unique()
fig = make_subplots(rows=4, cols=4, subplot_titles=list_of_substances)
x_loc=1
y_loc=1

for value in list_of_substances:
    drug_name = cleaned_data["substance"] == value
    fig.add_trace(
        go.Box(y=clean_no_duplicates[drug_name]["price per gram"]),
        row=x_loc, col=y_loc
    )
    x_loc +=1
    if x_loc==5:
        y_loc+=1
        x_loc=1

fig.show()

The amount of outliers we have is clearly huge.  Let's run a function to try and remove those and replicate that work again

In [0]:
# lets remove price outliers

from scipy.stats import zscore

list_of_filtered = []
for value in list_of_substances:
    drug_name = cleaned_data["substance"] == value
    drug_df = clean_no_duplicates[drug_name]
    drug_df["z score"] = zscore(drug_df["price"])
    z_score_filter = abs(drug_df["z score"]) <3
    new_df = drug_df[z_score_filter]
    list_of_filtered.append(new_df)

clean_no_outliers = pd.concat(list_of_filtered)

In [0]:
list_of_substances = cleaned_data["substance"].unique()
fig = make_subplots(rows=4, cols=4, subplot_titles=list_of_substances)
x_loc=1
y_loc=1

for value in list_of_substances:
    drug_name = clean_no_outliers["substance"] == value
    fig.add_trace(
        go.Box(y=clean_no_outliers[drug_name]["price"], boxpoints=False),
        row=x_loc, col=y_loc
    )
    x_loc +=1
    if x_loc==5:
        y_loc+=1
        x_loc=1

fig.update_layout(showlegend=False)
fig.show()


Let's try only adverts that have actually made a sale.


In [0]:
sale_filter = cleaned_data["Extracted sold"] > 0

made_sale = cleaned_data[sale_filter]
list_of_substances = made_sale["substance"].unique()

fig = make_subplots(rows=2, cols=7)
x_loc=1
y_loc=1

for value in list_of_substances:
    drug_name = made_sale["substance"] == value
    fig.add_trace(
        go.Box(y=made_sale[drug_name]["price"], boxpoints=False, name=value),
        row=x_loc, col=y_loc
    )
    y_loc +=1
    if y_loc==8:
        x_loc+=1
        y_loc=1

fig.update_layout(title="Average Price of Adverts with at least 1 Sale, by Substance", showlegend=False, width=1000)
fig.show()

Finally, let's replicate that using estimated Price per Gram

In [0]:
sale_filter = cleaned_data["Extracted sold"] > 0

made_sale = cleaned_data[sale_filter]
list_of_substances = made_sale["substance"].unique()

fig = make_subplots(rows=2, cols=7)
x_loc=1
y_loc=1

for value in list_of_substances:
    drug_name = made_sale["substance"] == value
    fig.add_trace(
        go.Box(y=made_sale[drug_name]["price per gram"], boxpoints=False, name=value),
        row=x_loc, col=y_loc
    )
    y_loc +=1
    if y_loc==8:
        x_loc+=1
        y_loc=1

fig.update_layout(title="Average Price per Gram of Adverts with at least 1 Sale, by Substance", showlegend=False, width=1000)
fig.show()

In [0]:
fig, axs = plt.subplots(4, 4)
fig.subplots_adjust(wspace=0.5,hspace=1)
fig.suptitle("Mean Advert Price")

x=0
y=0
for value in list_of_substances:
    drug_name = cleaned_data["substance"] == value
    cleaned_data[drug_name].pivot_table(values="price", index="date",aggfunc=np.mean).plot(ax=axs[x,y], legend=False, figsize=[8,8])
    axs[x,y].set_title(value)
    x +=1
    if x==4:
        y+=1
        x=0

Let's replicate that, but using my estimated price per gram and the median instead

In [0]:
list_of_substances = cleaned_data["substance"].unique()
fig, axs = plt.subplots(4, 4)
fig.subplots_adjust(wspace=0.5,hspace=1)
fig.suptitle("Median Advert Price per Gram")

x=0
y=0
for value in list_of_substances:
    drug_name = cleaned_data["substance"] == value
    cleaned_data[drug_name].pivot_table(values="price per gram", index="date",aggfunc=np.median).plot(ax=axs[x,y], legend=False, figsize=[8,8])
    axs[x,y].set_title(value)
    x +=1
    if x==4:
        y+=1
        x=0